# Simple Exponential Smoothing Metrics

This notebook runs exponential smoothing model for various smoothing constants ranging from 0.0 to 1.0 and save evaluation results.

In [1]:
import module.util_functions as utf
import module.constants as const
from tqdm import tqdm
import os
import pandas as pd
import numpy as np

# load sales data for all stores
store_sales = utf.load_ts_data('store_sales.csv')

In [2]:
from module.es_model import ExponentialSmoothing

store_list = []         # a list of stores
alpha_list = []         # a list of smoothing constants
mape_list = []          # a list of MAPE
mad_list = []           # a list of MAD
rmse_list = []          # a list of RMSE
r_squared_list = []     # a list of R-Squared

# iterate through a list of stores
for store_num in tqdm(store_sales.Store.unique()):
    # get sales data for a given store
    data = utf.get_store_sales(store_sales, store_num)
    
    # iterate through various alpha values ranging from 0.0 to 1.0
    for smoothing_constant in np.arange(0.0, 1.01, 0.01):
        smoothing_constant = np.round(smoothing_constant, 2)
        
        # fit an exponential smoothing model and predict next sales value using Rolling Forecast approach
        model = ExponentialSmoothing(smoothing_constant)
        forecasts, _, _ = utf.make_forecast(model, data)

        # evaluate model and save result
        mape, mad, rmse, r_squared = utf.compute_metrics(data.tail(const.TEST_SIZE)[const.STORE_OBSERVE], forecasts)
        mape_list.append(mape)
        mad_list.append(mad)
        rmse_list.append(rmse)
        alpha_list.append(smoothing_constant)
        r_squared_list.append(r_squared)
        store_list.append(store_num)

100%|██████████| 45/45 [14:34<00:00, 19.44s/it]


In [3]:
# create a metric dataframe and save as a csv file
metrics = pd.DataFrame({'Store': store_list, 'Alpha': alpha_list, 'MAPE': mape_list, 
                        'MAD': mad_list, 'RMSE': rmse_list, 'R-Squared': r_squared_list})
metrics['Model'] = ['ES'] * metrics.shape[0]
metrics.to_csv(os.path.join('results', 'es_metrics.csv'), index=False)